In [1]:
import numpy as np
from torchvision import transforms
import torch
import torch.nn as nn
from torch import optim
from torchvision.datasets import MNIST
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
import time
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')

In [15]:
class CustomDataset(Dataset): 
    def __init__(self):
        self.x_data = np.random.randint(1,100,size=(10000,1))
        self.x_data = np.concatenate([self.x_data, self.x_data**2], axis = -1)
        self.y_data = self.x_data[:,-1]*2 + 1
        self.y_data = self.y_data.reshape(-1,1)

  # 총 데이터의 개수를 리턴
    def __len__(self): 
        return len(self.x_data)

  # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
    def __getitem__(self, idx): 
        x = torch.FloatTensor(self.x_data[idx])
        y = torch.FloatTensor(self.y_data[idx])
        return x, y
    
train_dataset = CustomDataset()
print(train_dataset.__getitem__(0))
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size = batch_size, drop_last=True)
total_len = len(train_loader)

(tensor([  71., 5041.]), tensor([10083.]))


In [21]:
class MNISTNet(nn.Module):
    
    def __init__(self, batch_size):
        super(MNISTNet, self).__init__()
        self.linear1 = nn.Linear(2,1)
        
    def forward(self, X):
        return self.linear1(X)

def train(model, train_set, optimizer1, criterion, total_len):
    total_cost = 0
    l = np.linspace(0,total_len,10)
    l_idx = 0
    for batch_idx, (data, target) in enumerate(train_set):
        if batch_idx >= l[l_idx]:
            print('-', end='')
            l_idx+=1
        optimizer1.zero_grad()
        output = model(data.to(device))
        loss = criterion(output, target.to(device))
        total_cost += loss
        loss.backward()
        optimizer1.step()
        
    print('>', end='')
    print('    loss :',total_cost.item(), end='\n')

model = MNISTNet(batch_size)

In [30]:

optimizer1 = optim.Adam(model.parameters(), lr = 0.001)
#optimizer2 = optim.Adam(model.linear2(), lr = 0.005)
criterion = nn.MSELoss()
for epoch in range(10):
    train(model, train_loader, optimizer1, criterion, total_len)

--------->    loss : 85.71954345703125
--------->    loss : 1.232811689376831
--------->    loss : 1.174411416053772
--------->    loss : 1.1042791604995728
--------->    loss : 0.9587013125419617
--------->    loss : 0.8840550780296326
--------->    loss : 0.7667635679244995
--------->    loss : 0.8660120964050293
--------->    loss : 0.7234184741973877
--------->    loss : 1.2757034301757812


In [31]:
model.linear1.weight

Parameter containing:
tensor([[0.0056, 1.9999]], requires_grad=True)

In [308]:
import torch.nn.functional as F

model.eval()
x = 30000
y = 32051
output = model(torch.FloatTensor([x,y]))
print(output, np.sqrt(x**2+y**2))

tensor([4401.1890], grad_fn=<AddBackward0>)